In [82]:
!pip3 install -U datapane

     |████████████████████████████████| 1.2MB 6.6MB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
     |████████████████████████████████| 112kB 24.0MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 174kB 23.1MB/s 
     |████████████████████████████████| 552kB 24.0MB/s 
     |████████████████████████████████| 8.9MB 18.9MB/s 
     |████████████████████████████████| 13.2MB 304kB/s 
     |████████████████████████████████| 5.5MB 40.0MB/s 
     |████████████████████████████████| 20.7MB 1.6MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 645kB 35.0MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 552kB 35.9MB/s 
     |████████████████████████████████| 2.2MB 37.9MB/s 
     |████████████████████████████████| 112kB 35.1MB/s 
  Created wheel for micawber: filename=micawber-0.5.2-cp3

In [1]:
from datetime import date
today = date.today()
print(today)

2021-02-22


In [2]:
!pip install --upgrade pandasql

Requirement already up-to-date: pandasql in /usr/local/lib/python3.6/dist-packages (0.7.3)


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [4]:
# Set datapane.com token here

import datapane as dp
dp.login(token="xxxxx")

Connected successfully to https://datapane.com as nekrut


In [5]:
# Read variant data
datasets = {
    'boston_var': 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/bos_by_sample_processed.tsv.gz',
    'boston_sites': 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/bos_by_var_processed.tsv.gz',
    'pre_var':    'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/cog-pre_by_sample_processed.tsv.gz',
    'pre_sites':    'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/cog-pre_by_var_processed.tsv.gz',   
    'post_var':    'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/cog-post_by_sample_processed.tsv.gz',
    'post_sites':    'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/processed/cog-post_by_var_processed.tsv.gz' 
}

cutoffs = {
    'boston' : 3,
    'pre'    : 2,
    'post'   : 2
}

In [6]:
colors = {'Synonymous':'green','Non-synonymous':'orange','Stop':'red','Non-coding':'blue', 'Indel':'magenta'}

In [7]:
dfs = {}
for dataset in datasets:
    dfs[dataset] = pd.read_csv(datasets[dataset],sep='\t')
    dfs[dataset]['study'] = dataset.split('_')[0]

In [8]:
for key in cutoffs:
    s = dfs['{}_sites'.format(key)]
    v = dfs['{}_var'.format(key)]
    v = pysqldf('select v.*,"countunique(Sample)" from v left join s on v.POS = s.POS and v.change = s.change ')
    #s = s[s['countunique(Sample)']>=cutoffs[key]]
    #v = v[v['countunique(Sample)']>=cutoffs[key]]
    v = pd.concat( [ v[( v['countunique(Sample)']>=cutoffs[key]) & ( v['AF']<=.5 ) ] ,      v[v['AF']>0.5] ] )
    s = pd.concat( [ s[( s['countunique(Sample)']>=cutoffs[key]) & ( s['max(AF)']<=.5 ) ] , s[s['max(AF)']>0.5] ] )
    af_stat = v.groupby(['POS','change'])['AF'].describe()
    af_stat['CoV']=af_stat['std']/af_stat['mean']
    af_stat.fillna(0,inplace=True)
    s = pysqldf('select s.*, CoV,mean from s left join af_stat on s.POS = af_stat.POS and s.change = af_stat.change ')
    dfs['{}_var'.format(key)] = v
    dfs['{}_sites'.format(key)] = s
    s['frac']=(s['countunique(Sample)']/v['Sample'].nunique())
del(v)
del(s)

In [9]:
var = pd.concat( [ dfs[x+'_var'] for x in cutoffs ] )
sites = pd.concat( [ dfs[x+'_sites'] for x in cutoffs ] )

In [10]:
for func in var['FUNCLASS'].unique():
    var.loc[var['FUNCLASS'] == func, 'color'] = colors[func]
    sites.loc[sites['FUNCLASS'] == func, 'color'] = colors[func]

In [11]:
for x in [ dfs[x] for x in datasets ]:
    del(x)

In [12]:
annot = {
    'start': [    265,    805,   2719,   8554,   10054,  10972,  11842,  12091,  12685,   13024,  13441,      16236,  18039,   19620,    20658,   13441,    21562,   25392, 26244, 26522, 27201,   27393,   27755,  27893, 28273, 29557],
    'end':   [    805,   2719,   8554,  10054,   10972,  11842,  12091,  12685,  13024,   13441,  16236,      18039,  19620,   20658,    21552,   13480,    25384,   26220, 26472, 27191, 27387,   27759,   27887,  28259, 29533, 29674 ], 
    'func':  ['leader', 'nsp2', 'nsp3', 'nsp4', '3Cpro', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'RdRp', 'helicase', 'ExoN', 'endoR', 'MethTr', 'nsp11', 'S', 'orf3a',   'E',   'M', 'orf6', 'orf7a', 'orf7b', 'orf8',  'N', 'orf10'], 
    }
annot = pd.DataFrame.from_dict(annot)[['start','end','func']].sort_values(by=['start']).reset_index()
annot['top']= annot.index % 2
annot['bottom'] = annot['top']-1
annot.loc[annot['top'] == 0, 'color'] = 'red'
annot.loc[annot['top'] != 0, 'color'] = 'blue'

In [13]:
annot.head( )

,index,start,end,func,top,bottom,color
0,0,265,805,leader,0,-1,red
1,1,805,2719,nsp2,1,0,blue
2,2,2719,8554,nsp3,0,-1,red
3,3,8554,10054,nsp4,1,0,blue
4,4,10054,10972,3Cpro,0,-1,red


In [14]:
# Variants of concern
voc_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/voc/voc.tsv.gz'
voc = pd.read_csv(voc_url, sep='\t', header=None, names=['set','aa','position'])

In [15]:
voc.head()

,set,aa,position
0,B1351,P71L,26454
1,B1351,T205I,28885
2,B1351,K1655N,5227
3,B1351,D80A,21799
4,B1351,D215G,22204


In [16]:
# Sites under selection URL
sel_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/selection/selection.tsv.gz'
sel = pd.read_csv(sel_url,sep='\t')

In [17]:
pysqldf('select count(*) from sel')

,count(*)
0,1399


In [18]:
sel = pysqldf("select * from sel where fel_p <= 0.0001 or meme_p <= 0.0001")

In [19]:
pysqldf('select count(*) from sel')

,count(*)
0,306


In [20]:
var_sel = pysqldf('select var.*, fel_p,meme_p from var join sel on var.POS >= sel.pos and var.POS <= sel.pos+3 and fel_p is not NULL and meme_p is not NULL')

In [21]:
var.head()

,Sample,POS,FILTER,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,min(AF),max(AF),countunique(change),countunique(FUNCLASS),change,type,study,countunique(Sample),color
1,SRR11953670,15070,PASS,T,A,114,0.087719,30,"39,64,10,1",MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,cTt/cAt,L544H,RdRp,0.006390,0.296296,1,1,T>A,SNP,boston,622,orange
10,SRR11953673,3466,PASS,G,T,1356,0.050147,132,"439,838,61,18",HIGH,Stop,STOP_GAINED,ORF1ab,Gga/Tga,G250*,nsp3,0.004189,0.117188,1,1,G>T,SNP,boston,504,red
11,SRR11953673,4228,PASS,A,C,1234,0.054295,154,"379,782,59,14",MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,Aca/Cca,T504P,nsp3,0.005070,0.136364,1,1,A>C,SNP,boston,534,orange
12,SRR11953673,12412,PASS,A,C,1096,0.058394,138,"378,650,57,11",MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,Aac/Cac,N108H,nsp8,0.004820,0.428571,1,1,A>C,SNP,boston,590,orange
13,SRR11953673,12925,PASS,A,AC,988,0.051619,117,"432,505,48,3",HIGH,Indel,FRAME_SHIFT,ORF1ab,aaa/aCaa,K81T?,nsp9,0.004034,0.253731,1,1,A>AC,Indel,boston,631,magenta


In [22]:
sites_sel = pysqldf('select sites.* from sites join sel on sites.POS >= sel.pos and sites.POS <= sel.pos+3 and sites.fel_p is not NULL and sites.meme_p is not NULL')

In [24]:
sites_sel.head()

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel_p,meme_p,freq,B1351,P1,B117,BLOOM,A321,type,study,CoV,mean,frac,color
0,1820,G,A,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gGt/gAt,G339D,nsp2,115,0.001327,0.081818,SRR11954207,"SRR12733283,SRR12733290,SRR12733308,SRR1273331...","0.001803,0.003022,0.002412,0.002713,0.004607,0...",G>A,3.237843e-11,1.121477e-10,0.003398,None,None,None,None,None,SNP,boston,0.000000,0.081818,0.179969,orange
1,1911,C,T,LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,tcC/tcT,S369,nsp2,5,0.002274,0.180157,SRR12733576,"SRR12733576,SRR12733807,SRR12733977,SRR1195401...","0.180157,0.007515,0.002274,0.008137,0.005625",C>T,6.704042e-06,6.666667e-01,0.005669,None,None,None,None,None,SNP,boston,0.000000,0.180157,0.007825,green
2,3410,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gCt/gTt,A231V,nsp3,3,0.002208,0.209605,SRR11953945,"SRR11954171,SRR11954145,SRR11953945","0.030234,0.002208,0.209605",C>T,4.317519e-05,9.879708e-05,0.002390,None,None,None,None,None,SNP,boston,0.000000,0.209605,0.004695,orange
3,6637,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,Cat/Tat,H1307Y,nsp3,3,0.026178,0.063889,"SRR11954150,SRR11954196","SRR11954196,SRR11954150,SRR11953806","0.063889,0.056,0.026178",C>T,3.454074e-04,7.120791e-04,0.000986,None,None,None,None,None,SNP,boston,0.093059,0.059944,0.004695,orange
4,11073,C,CT,HIGH,Indel,FRAME_SHIFT,ORF1ab,ttg/ttTg,L37F?,nsp6,12,0.022663,0.125000,"SRR11954125,SRR11954279,SRR12733354,SRR1273349...","SRR12733354,SRR12733406,SRR12733492,SRR1273352...","0.051471,0.0452,0.055645,0.048352,0.047045,0.0...",C>CT,2.413795e-05,6.666667e-01,0.002636,None,None,None,None,None,Indel,boston,0.129009,0.055716,0.018779,magenta


In [25]:
sites_sel['frac'] = sites_sel['frac']*100

In [26]:
pysqldf('select study,count(*) as N from sites_sel group by study')

,study,N
0,boston,84
1,post,441
2,pre,199


In [27]:
pysqldf('select study,count(*) as N from sites_sel where "max(AF)" <= .8 group by study')

,study,N
0,boston,13
1,post,155
2,pre,130


In [28]:
sites_sel.head()

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel_p,meme_p,freq,B1351,P1,B117,BLOOM,A321,type,study,CoV,mean,frac,color
0,1820,G,A,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gGt/gAt,G339D,nsp2,115,0.001327,0.081818,SRR11954207,"SRR12733283,SRR12733290,SRR12733308,SRR1273331...","0.001803,0.003022,0.002412,0.002713,0.004607,0...",G>A,3.237843e-11,1.121477e-10,0.003398,None,None,None,None,None,SNP,boston,0.000000,0.081818,17.996870,orange
1,1911,C,T,LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,tcC/tcT,S369,nsp2,5,0.002274,0.180157,SRR12733576,"SRR12733576,SRR12733807,SRR12733977,SRR1195401...","0.180157,0.007515,0.002274,0.008137,0.005625",C>T,6.704042e-06,6.666667e-01,0.005669,None,None,None,None,None,SNP,boston,0.000000,0.180157,0.782473,green
2,3410,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gCt/gTt,A231V,nsp3,3,0.002208,0.209605,SRR11953945,"SRR11954171,SRR11954145,SRR11953945","0.030234,0.002208,0.209605",C>T,4.317519e-05,9.879708e-05,0.002390,None,None,None,None,None,SNP,boston,0.000000,0.209605,0.469484,orange
3,6637,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,Cat/Tat,H1307Y,nsp3,3,0.026178,0.063889,"SRR11954150,SRR11954196","SRR11954196,SRR11954150,SRR11953806","0.063889,0.056,0.026178",C>T,3.454074e-04,7.120791e-04,0.000986,None,None,None,None,None,SNP,boston,0.093059,0.059944,0.469484,orange
4,11073,C,CT,HIGH,Indel,FRAME_SHIFT,ORF1ab,ttg/ttTg,L37F?,nsp6,12,0.022663,0.125000,"SRR11954125,SRR11954279,SRR12733354,SRR1273349...","SRR12733354,SRR12733406,SRR12733492,SRR1273352...","0.051471,0.0452,0.055645,0.048352,0.047045,0.0...",C>CT,2.413795e-05,6.666667e-01,0.002636,None,None,None,None,None,Indel,boston,0.129009,0.055716,1.877934,magenta


In [29]:
sites_sel = sites_sel.rename(columns={"countunique(Sample)": "samples"})

In [30]:
sites_sel.head()

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,samples,min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel_p,meme_p,freq,B1351,P1,B117,BLOOM,A321,type,study,CoV,mean,frac,color
0,1820,G,A,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gGt/gAt,G339D,nsp2,115,0.001327,0.081818,SRR11954207,"SRR12733283,SRR12733290,SRR12733308,SRR1273331...","0.001803,0.003022,0.002412,0.002713,0.004607,0...",G>A,3.237843e-11,1.121477e-10,0.003398,None,None,None,None,None,SNP,boston,0.000000,0.081818,17.996870,orange
1,1911,C,T,LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,tcC/tcT,S369,nsp2,5,0.002274,0.180157,SRR12733576,"SRR12733576,SRR12733807,SRR12733977,SRR1195401...","0.180157,0.007515,0.002274,0.008137,0.005625",C>T,6.704042e-06,6.666667e-01,0.005669,None,None,None,None,None,SNP,boston,0.000000,0.180157,0.782473,green
2,3410,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,gCt/gTt,A231V,nsp3,3,0.002208,0.209605,SRR11953945,"SRR11954171,SRR11954145,SRR11953945","0.030234,0.002208,0.209605",C>T,4.317519e-05,9.879708e-05,0.002390,None,None,None,None,None,SNP,boston,0.000000,0.209605,0.469484,orange
3,6637,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,Cat/Tat,H1307Y,nsp3,3,0.026178,0.063889,"SRR11954150,SRR11954196","SRR11954196,SRR11954150,SRR11953806","0.063889,0.056,0.026178",C>T,3.454074e-04,7.120791e-04,0.000986,None,None,None,None,None,SNP,boston,0.093059,0.059944,0.469484,orange
4,11073,C,CT,HIGH,Indel,FRAME_SHIFT,ORF1ab,ttg/ttTg,L37F?,nsp6,12,0.022663,0.125000,"SRR11954125,SRR11954279,SRR12733354,SRR1273349...","SRR12733354,SRR12733406,SRR12733492,SRR1273352...","0.051471,0.0452,0.055645,0.048352,0.047045,0.0...",C>CT,2.413795e-05,6.666667e-01,0.002636,None,None,None,None,None,Indel,boston,0.129009,0.055716,1.877934,magenta


In [45]:
from bokeh.models import ColumnDataSource, Label, LabelSet,Range1d
from bokeh.layouts import column
import bokeh.io
import bokeh.plotting
from bokeh.models import Legend,ColumnDataSource
from bokeh.transform import linear_cmap
from math import pi
from bokeh.transform import factor_cmap
from bokeh.models import BoxAnnotation, Toggle,Range1d,Rect,Quad
from bokeh.plotting import figure, show
from bokeh.palettes import Set1_7,Reds
from bokeh.layouts import gridplot,column

bokeh.io.output_notebook()

def genome_plot(df,gene_coord,voc,ht,name):
    source_var = ColumnDataSource(df)
    gene_coord = ColumnDataSource(gene_coord)
    voc_coord = ColumnDataSource(voc)
    TOOLTIPS = [
                ("Position","@POS"),
                ("Codon","@CODON"),
                ("Amino acid",'@AA'),
                ("Samples with variant","@samples"),
                ("Gene","@TRID")
    ]
    p = figure(plot_height=ht,
        plot_width=1200,
        #y_range=df['Sample'].unique(),
        #tooltips = TOOLTIPS,
        x_axis_label = 'Genome coordinate',
        y_axis_label = 'Coefficient of Variation for AF',
        x_range=Range1d(start=0, end=30000,bounds=(0, 30000)),
        y_range=Range1d(start=0, end=2,bounds=(0, 2)),
        title=name,
        tooltips=TOOLTIPS
        )
    p.circle(x='POS',y='CoV',source=source_var,size="frac",alpha=.5,fill_color='color')
    #p.xaxis.ticker = df['POS']
    p.xaxis.major_label_orientation = pi/2
    p_labels = LabelSet(x='POS', y='CoV', text='POS', level='glyph',
              x_offset=5, y_offset=5, source=source_var, render_mode='canvas',angle=np.pi/6,text_font_size='6pt')
    p.add_layout(p_labels)
    q = figure(
        x_range=p.x_range,
        plot_height=100,
        plot_width=1200,
        y_range=voc['set'].unique(),
        tools='save')
#glyph_q = Rect(x='position',y='set')
    q.circle(x='position',y='set',source=voc_coord,color='red',size=2)
#p.add_glyph(voc_coord,glyph_q)


# Figure for annotation track
    r = figure(
        x_range=p.x_range,
        plot_height=100,
        plot_width=1200,
        #**plot_options,
        y_axis_label='Genes',
        x_axis_label='Position in genome',
        y_range=Range1d(start=-1, end=2,bounds=(-1, 2)),
        tools='save'
    )

    r_labels = LabelSet(x='start', y='bottom', text='func', level='glyph',
              x_offset=0, y_offset=0, source=gene_coord, render_mode='canvas',text_font_size='7pt', angle=np.pi/2  )

    genes = Quad(left="start", bottom='bottom', right='end', top='top',line_color='color',fill_color='color',fill_alpha=.5)
    r.add_glyph(gene_coord, genes)
    r.add_layout(r_labels)
    r.yaxis.visible = False
    r.ygrid.grid_line_color = None

    show(column(p,q,r))
    return(p,q,r)

In [46]:
plots = []
for study in sites_sel['study'].unique():
    p,q,r = genome_plot(sites_sel[sites_sel['study']==study],annot,voc,400,study.capitalize())
    plots.append(p)
    

In [47]:
# Fixme -> needs to be polulated dynamically
r = dp.Report(
    dp.Group(
        dp.Plot(plots[0]),
        dp.Plot(plots[1]),
        dp.Plot(plots[2])
    )
)
r.publish(name='variants versus sites under selection all', open=True)

Publishing report and associated data - please wait..
Report successfully published at https://datapane.com/u/nekrut/reports/variants-versus-sites-under-selection-all/


In [48]:
plots = []
for study in sites_sel['study'].unique():
    p,q,r = genome_plot(sites_sel[ ( sites_sel['study']==study ) & ( sites_sel['max(AF)']<= 0.8 )],annot,voc,400,'{}, Max AF < 0.8'.format(study.capitalize()))
    plots.append(p)

In [49]:
# Fixme -> needs to be polulated dynamically
r = dp.Report(
    dp.Group(
        dp.Plot(plots[0]),
        dp.Plot(plots[1]),
        dp.Plot(plots[2])
    )
)
r.publish(name='variants versus sites under selection < 80%', open=True)

Publishing report and associated data - please wait..
Report successfully published at https://datapane.com/u/nekrut/reports/variants-versus-sites-under-selection-80/


In [35]:
print(pysqldf('select study, meme_p,fel_p,POS, AA, CODON, REF, ALT, "min(AF)","max(AF)",CoV,mean,frac from sites_sel where POS > 11070 and POS < 11080 and fel_p is not NULL and meme_p is not NULL').to_markdown(index=False))

| study   |   meme_p |       fel_p |   POS | AA     | CODON       | REF   | ALT   |   min(AF) |   max(AF) |      CoV |      mean |      frac |
|:--------|---------:|------------:|------:|:-------|:------------|:------|:------|----------:|----------:|---------:|----------:|----------:|
| boston  | 0.666667 | 2.41379e-05 | 11073 | L37F?  | ttg/ttTg    | C     | CT    |  0.022663 |  0.125    | 0.129009 | 0.0557156 |  1.87793  |
| pre     | 0.666667 | 2.41379e-05 | 11072 | F34    | ttc/        | TC    | T     |  0.000199 |  0.160824 | 0        | 0.160824  |  5.5666   |
| pre     | 0.666667 | 2.41379e-05 | 11073 | L37F?  | ttg/ttTg    | C     | CT    |  0.003119 |  0.222329 | 0.471049 | 0.065703  | 18.2903   |
| pre     | 0.666667 | 2.41379e-05 | 11073 | L37F?  | ttg/ttTTg   | C     | CTT   |  0.001736 |  0.207648 | 0.622544 | 0.14418   | 12.7237   |
| pre     | 0.666667 | 2.41379e-05 | 11073 | L37FL  | ttg/ttTTTg  | C     | CTTT  |  0.002071 |  0.401176 | 0.660965 | 0.273397  |  5.5666   |

In [32]:
pysqldf('select * from sites_sel where POS = 23557')

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel_p,meme_p,freq,B1351,P1,B117,BLOOM,A321,type,study,CoV,mean,frac,color


In [33]:
pysqldf('select * from sel where pos >= 11070 and pos <= 11080')

,pos,gene,site,fel_p,fel_a,fel_b,meme_p,meme_a,meme_b_p,meme_b_m,freq,REF,ALT,codon,slip
0,11071,ORF1a,3603,2.413795e-05,10.621234,0.000000,6.666667e-01,10.618832,109.218819,0.0,0.002636,302683,800,TTC,0
1,11080,ORF1a,3606,3.499813e-10,0.000000,23.177973,1.137479e-09,0.000000,23.117339,0.0,0.060119,285236,18245,TTG,0


In [34]:
print(pysqldf('select *  from sites_sel where POS = 25842 or POS = 22254').to_markdown(index=False))

|   POS | REF   | ALT   | IMPACT   | FUNCLASS       | EFFECT                | GENE   | CODON   | AA    | TRID   |   countunique(Sample) |   min(AF) |   max(AF) | SAMPLES(above-thresholds)                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | SAMPLES(all)                                                                                                                                                                                                                                                                                                                                                        